In [28]:
import json
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense
from numpy import asarray, zeros

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
NUM_SENTENCES = 10000 # Use only the first 10,000 records.
MAX_NUM_WORDS = 10000 # Use 10,000 words for tokenizing
MAX_SENT_LEN = 50

EMBEDDING_SIZE = 100

LSTM_NEURONS = 100

BATCH_SIZE = 64
EPOCHS = 5

In [4]:
# To upload the necessary files for the first time
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [3]:
inputs = []
outputs_i = []
outputs = []

data_file = open("jsonDataset.txt", encoding="utf-8")
dataset_json_str = ""

count = 0
for line in data_file:
  count += 1
  if count > NUM_SENTENCES:
      break
  dataset_json_str += line.rstrip()
  
dataset_json_str = dataset_json_str[:-1] + "]}"
dataset_json = json.loads(dataset_json_str)

for i in range(0, NUM_SENTENCES):
  ip = dataset_json["dataset"][i]["translation"]["ot"]
  temp_op = dataset_json["dataset"][i]["translation"]["tr"]
  op_i = '<sos> '+temp_op
  op = temp_op+' <eos>'
  inputs.append(ip)
  outputs_i.append(op_i)
  outputs.append(op)

print('Total inputs =', len(inputs))
print('Total intermediate outputs =', len(outputs_i))
print('Total outputs =', len(outputs))
print('\nSample:')
print(inputs[100])
print(outputs_i[100])
print(outputs[100])

Total inputs = 10000
Total intermediate outputs = 10000
Total outputs = 10000

Sample:
hatta harbiye nazırı olan sakir paşa bu talimatı okuduktan sonra imzada tereddüt etmiş , anlaşılır anlaşılmaz bir tarzda , mührünü basmıştır .
<sos> hatta savaşişleri bakani olan sakir paşa bu talimati okuduktan sonra imzada duraksamiş , anlaşilir anlaşilmaz bir şekilde , mührünü basmiştir .
hatta savaşişleri bakani olan sakir paşa bu talimati okuduktan sonra imzada duraksamiş , anlaşilir anlaşilmaz bir şekilde , mührünü basmiştir . <eos>


In [4]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(inputs)

inputs_seq = input_tokenizer.texts_to_sequences(inputs)

inputs_word2index = input_tokenizer.word_index
print('Total unique words in input:', len(inputs_word2index))

inputs_numwords = len(inputs_word2index)+1

inputs_maxlen = max(len(s) for s in inputs_seq)
print('Length of longest sentence in input:', inputs_maxlen)

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

outputs_i_seq = output_tokenizer.texts_to_sequences(outputs_i)
outputs_seq = output_tokenizer.texts_to_sequences(outputs)

outputs_word2index = output_tokenizer.word_index
print('Total unique words in output:', len(outputs_word2index))

outputs_numwords = len(outputs_word2index)+1

outputs_maxlen = max(len(s) for s in outputs_seq)
print('Length of longest sentence in output:', outputs_maxlen)

Total unique words in input: 22306
Length of longest sentence in input: 97
Total unique words in output: 23404
Length of longest sentence in output: 109


In [5]:
encoder_input_sequences = pad_sequences(inputs_seq, maxlen=inputs_maxlen)
print('encoder_input_sequences shape:', encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(outputs_i_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_inputs_sequences shape:', decoder_input_sequences.shape)

decoder_output_sequences = pad_sequences(outputs_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_output_sequences shape:', decoder_output_sequences.shape)

encoder_input_sequences shape: (10000, 97)
decoder_inputs_sequences shape: (10000, 109)
decoder_output_sequences shape: (10000, 109)


In [6]:
embeddings_dict = dict()

glove_file = open('glove.6B.100d.txt', encoding='utf8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector

glove_file.close()

num_words = min(MAX_NUM_WORDS, len(inputs_word2index)+1)

embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

for word, index in inputs_word2index.items():
    embedding_vector = embeddings_dict.get(word)
    if index == num_words:
      break
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
encoder_embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=inputs_maxlen)
decoder_embedding_layer = Embedding(outputs_numwords, LSTM_NEURONS)

decoder_outputs_onehot = zeros((len(inputs), outputs_maxlen, outputs_numwords), dtype='float32')
print('decoder_outputs_onehot shape:', decoder_outputs_onehot.shape)

for i, d in enumerate(decoder_outputs_onehot):
    for t, w in enumerate(d):
        decoder_outputs_onehot[i, t, w] = 1

encoder_inputs = Input(shape=(inputs_maxlen,))
encoder_inputs_emb = encoder_embedding_layer(encoder_inputs)
encoder = LSTM(LSTM_NEURONS, return_state=True)
encoder_outputs, h, c = encoder(encoder_inputs_emb)
encoder_states = [h, c]

decoder_inputs = Input(shape=(outputs_maxlen,))
decoder_inputs_emb = decoder_embedding_layer(decoder_inputs)
decoder = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs_emb, initial_state=encoder_states)

output_dense_layer = Dense(outputs_numwords, activation='softmax')
outputs = output_dense_layer(decoder_outputs)